## Types of Clustering
There are many types of clustering algorithms of which here are the top 4 well-known ones:

* Connectivity-based Clustering
* Centroid-based Clustering
* Distribution-based Clustering
* Density-based Clustering

### Clustering Principles:
All clustering algorithms try to group data points based on similarities between the data. What does this actually mean?


It is often spoken of, in terms of **`inter-cluster heterogeneity`** and **`intra-cluster homogeneity`**. 

* `Inter-cluster heterogeneity`<br> This means that the clusters are as different from one another as possible. The characteristics of one cluster are very different from another cluster. This makes the clusters very stable and reliable.
* `Intra-cluster homogeneity`<br> This talks about how similar are the characteristics of all the data within the cluster. The more similar, the more cohesive is the cluster and hence more stable. 

* **Hence the objective of clustering is to maximise the inter-cluster distance (Inter-cluster heterogeneity) and minimise the intra-cluster distance (intra-cluster homogeneity )**



# K-Means Clustering

### STEP-1
*   #### Select the number of clusters you want to identify in your data
*   #### This is the `k` in kmeans

<img src='./notes/customer segmentation.png'>

### STEP-2
* #### Randomly select `k`  centroids -- cluster  centers
* #### These are the initial clusters

### STEP-3
* #### Measure the distance between data-points and  cluster-centers
* #### Use `Euclidean distance` for calculating the distance


### STEP-4
* #### Assign data-points to nearest cluster based on the computed distance


### STEP-5
* #### Calculate the mean of each cluster  [`UPDATING CLUSTER CENTROID`]
* #### Now repeat step-3 and 4
* #### Measure the distance between datapoints and cluster-mean
* #### Assign data-points to nearest cluster based on the distance calculated
* #### This process continues until the clusters dont actually change

<img src='./notes/customer segmentation - 1.png'>

## Kmeans are not that great 
* K-means produces unintuitive and possibly undesirable clusters.
* Usually by eyeballing we (humans) can identify better clustering than kmeans
* We can assess the quality of clustering by adding up the variation within each cluster
* Since k-means can't see the best clustering it's only option is to keep track of these clusters and their total variance and do the whole process over again with different starting points 
* Kmeans outputs the clustering that has the lowest variation
* **CAUTION :** Set the `k` wisely, kmeans will always find k clusters in the data 
* Kmeans put the data into the number of clusters we give it as input
* Whereas the Heirarchical clustering, tells us, pairwise, what two things are most similar

## If you tell it to give you 4 clusters, it'll give you four clusters
<img src='./notes/customer segmentation - 2.png'>


## How do you figure out what value to use for `K`
* Domain knowledge
* Trail & Error 
    * Elbow plot <br>
    You can find `k` by finding the elbow in elbow-plot<br>
    X-axis : different values of `k`<br>
    Y-axis : reduction in variation <br>
    You can observe a huge reduction in variation when `k` = # clusters

<img src='./notes/Elbow-plot.png'>

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from scipy.linalg import eigh

from sklearn.cluster import KMeans, kmeans_plusplus
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs

from sklearn.metrics import silhouette_samples, silhouette_score

## Demonstration of k-means 


#### Data generation
* The function `make_blobs` generates `isotropic (spherical) gaussian blobs`. 

* To obtain `anisotropic (elliptical) gaussian blobs` one has to define a linear transformation.

In [2]:
n_samples = 1500
random_state = 170
transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]]

# spherical blobs [1500, 2]
X, y = make_blobs(n_samples=n_samples, random_state=random_state, n_features=2, centers=3)

# elliptical blobs  [1500, 2] 
# matrix multiplication
X_aniso = np.dot(X, transformation)  # Anisotropic blobs


# Unequal variance
X_varied, y_varied = make_blobs(
    n_samples=n_samples, cluster_std=[1.0, 2.5, 0.5], random_state=random_state, n_features=2, centers=3
)  


# Unevenly sized blobs
X_filtered = np.vstack(
    (X[y == 0][:500], X[y == 1][:100], X[y == 2][:10])
)  

y_filtered = [0] * 500 + [1] * 100 + [2] * 10

#### Plot the data

In [3]:
data = {
    "Mixture of Gaussian Blobs" : (X, y),
    "Anisotropically Distributed Blobs" : (X_aniso, y),
    "Unequal Variance" : (X_varied, y_varied),
    "Unevenly Sized Blobs" : (X_filtered, y_filtered)
}

In [10]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
ax = ax.ravel()

keys = list(data.keys())

for i, frame in enumerate(ax):
    points, labels = data[keys[i]]
    frame.scatter(points[:,0], points[:,1], c=labels)
    frame.set(title=keys[i])
    frame.set_xticks([])
    frame.set_yticks([])

plt.tight_layout()

<img src='./plots/clusters.png'>

## How do we set the `k` in kmeans ?
#### How do we set the number of clusters or How do you figure out what value to use for `K`
* If we ask for `k` clusters the Kmeans will put the data into `k` clusters 
* **CAUTION** : `Non-optimal number of clusters`:<br>
In a real world setting there is no uniquely defined true number of clusters. An appropriate number of clusters has to be decided from data-based criteria and knowledge of the intended goal.

In [4]:
def plot_clusters(ax, title, X, y):
    ax.scatter(X[:, 0], X[:, 1], c=y)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(title=title)




fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(12, 6))
ax = ax.ravel()

plot_clusters(ax[0], 'True clustering', X, y)

for i in [1,2,3,4,5]:
    y_preds = KMeans(n_clusters=i, n_init='auto', random_state=random_state).fit_predict(X, y)
    plot_clusters(ax[i], f'kmeans clustering with k={i}', X, y_preds)




<img src='./plots/kmeans-clustering-and-parameter-k.png'>

### Possible solutions : silhouette analysis on KMeans clustering
* Selecting the number of clusters with silhouette analysis on KMeans clustering.

A higher Silhouette Coefficient score relates to a model with better defined clusters. 

The Silhouette Coefficient is defined for each sample and is composed of two scores:
* `a`: The mean distance between a sample and all other points in the same class.
* `b`: The mean distance between a sample and all other points in the next nearest cluster.

The Silhouette Coefficient s for a single sample is then given as:
* `(b - a) / max(a, b)`

The Silhouette Coefficient for a set of samples is given as the mean of the Silhouette Coefficient for each sample.

#### Advantages
* The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.

* The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.

In [5]:


def silhouette_plot(k, X, y):
    y_preds = KMeans(n_clusters=k, n_init='auto', random_state=random_state).fit_predict(X, y)

    score = silhouette_score(X, y_preds)

    # score for each sample
    sample_score = silhouette_samples(X, y_preds)

    # style : adding indent between bars
    y_low= 10
    # style : generate colors
    colors__for_truth = plt.cm.nipy_spectral(y.astype(float) / len(np.unique(y)))
    colors__for_prediction = plt.cm.nipy_spectral(y_preds.astype(float) / k)

    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    ax[0].scatter(X[:, 0], X[:, 1], c=colors__for_truth)
    ax[0].set(title='True cluster')

    ax[1].scatter(X[:, 0], X[:, 1], c=colors__for_prediction)
    ax[1].set(title=f'Kmeans clustering with k = {k}')

    for cluster in range(k):
        score_of_samples_in_the_cluster = sample_score[y_preds == cluster]
        # sort the score
        score_of_samples_in_the_cluster.sort()

        # size of cluster
        cluster_size = score_of_samples_in_the_cluster.shape[0]
        # style : adding indent between bars
        y_high = cluster_size + y_low

        # style : generate colors
        color = plt.cm.nipy_spectral(cluster/k)

        y_axis = np.arange(y_low, y_high)
        ax[2].fill_betweenx(y_axis, 0, score_of_samples_in_the_cluster, facecolor=color)

        # style : adding indent between bars
        y_low = y_high + 10

        ax[2].set(title=f'Silhouette analysis\nscore this clustering = {score:0.2f}')

    # Silhouette score for the samples 
    ax[2].axvline(x=score, c='r')





### When `k` = 2 , the obtained clusters have scores above average score
### From the thickness of the silhouette plot the cluster size can be visualized. 

In [12]:
silhouette_plot(2, X, y)

<img src='./plots/silhouette_plot_kmeans_k_2.png'>

### When `k` = 3 , all the plots are more or less of similar thickness 
### Cluster scores are above the average 
### Looks perfect

In [13]:
silhouette_plot(3, X, y)

<img src='./plots/silhouette_plot_kmeans_k_3.png'>

#### `k`=4 is a bad pick for the given data due to the presence of clusters with below average silhouette scores and also due to wide fluctuations in the size of the silhouette plots. 

In [14]:
silhouette_plot(4, X, y)

<img src='./plots/silhouette_plot_kmeans_k_4.png'>

### When `k`=5 , there is wide fluctuations in the size of the silhouette plots but all cluster have score above the average

In [15]:
silhouette_plot(5, X, y)

<img src='./plots/silhouette_plot_kmeans_k_5.png'>

## "Anisotropically Distributed Blobs"  | `elliptical gaussian blobs`
k-means consists of minimizing sample’s euclidean distances to the centroid of the cluster they are assigned to. As a consequence, **k-means is more appropriate for clusters that are isotropic and normally distributed** (i.e. `spherical gaussians`).

In [46]:
y_preds = KMeans(n_clusters=3, n_init='auto', random_state=random_state).fit_predict(X_aniso, y)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax = ax.ravel()

plot_clusters(ax[0], 'The data is elliptical gaussian distributed', X_aniso, y)

plot_clusters(ax[1], 'Kmeans algorithm is appropriate for spherical gaussians\nits not appropriate to use here', X_aniso, y_preds)

<img src='./plots/kmeans-are-appropriate-for-spherical-gaussian-not-for-elliptical-gaussian.png'>

## Anisotropic and unequal variances are real limitations of the k-means algorithm

So instead of kmeans lets use  GaussianMixture, which also assumes gaussian clusters but does not impose any constraints on their variances. Notice that one still has to find the correct number of blobs

In [40]:
gauss_mix =  GaussianMixture(n_components=3, covariance_type='full')
y_preds = gauss_mix.fit_predict(X_aniso, y)


fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax = ax.ravel()
plot_clusters(ax[0], 'The data is elliptical gaussian distributed', X_aniso, y)
plot_clusters(ax[1], 'GaussianMixture', X_aniso, y_preds)


<img src='./plots/anisotropic_unequal_variance_and_gauss_mix.png'>

## Kmeans assumes cluster are spherical gaussian with same variance
* `Unequal variance`: k-means is equivalent to taking the maximum likelihood estimator for a “mixture” of k gaussian distributions with the **same variances but with possibly different means**.

In [50]:
y_preds = KMeans(n_clusters=3, n_init='auto', random_state=random_state).fit_predict(X_varied, y_varied)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
ax = ax.ravel()

plot_clusters(ax[0], 'Clusters have different variance', X_varied, y_varied)

plot_clusters(ax[1], 'Variance of clusters are not same\nHence kmeans clustering will not be robust', X_varied, y_preds)

<img src='./plots/clusters-have-different-variance-hence-kmeans-clustering-will-not-be-robust.png'>

## Several runs are recommended for sparse high-dimensional problems

* `Unevenly sized blobs` <br> there is no theoretical result about k-means that states that it requires similar cluster sizes to perform well, yet **minimizing euclidean distances does mean that the more sparse and high-dimensional the problem is, the higher is the need to run the algorithm with different centroid seeds to ensure a global minimal inertia**.

* `n_init` = ‘`auto`’ , default=10
Number of times the k-means algorithm is run with different centroid seeds. The final results is the best output of n_init consecutive runs in terms of inertia. Several runs are recommended for sparse high-dimensional problems.

In [11]:


fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(12, 8))
ax = ax.ravel()

plot_clusters(ax[0], 'Clusters have different size', X_filtered, y_filtered)

for i, j in enumerate([2, 5, 8, 10, 15]):
    y_preds = KMeans(n_clusters=3, n_init=j, random_state=random_state).fit_predict(X_filtered, y_filtered)
    plot_clusters(ax[i+1], f'kmeans with n_init={j}', X_filtered, y_preds)

fig.suptitle('Several runs are recommended for sparse high-dimensional problems', size=16)
plt.tight_layout()

<img src='./plots/Several-runs-are-recommended-for-sparse-high-dim-data.png'>

### To deal with unevenly sized blobs one can increase the number of random initializations. 
In this case we increase the n_init to avoid finding a sub-optimal local minimum. 

In [19]:
for i in [ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    y_preds = KMeans(n_clusters=3, n_init=i, random_state=random_state).fit_predict(X_filtered, y_filtered)
    n , size = np.unique(y_preds, return_counts=True)
    print(f'n_init: {i} , Number of elements assigned to each cluster: {size}')

n, true_size = np.unique(y_filtered, return_counts=True)
print(f'True number of elements assigned to each cluster : {true_size}')

n_init: 1 , Number of elements assigned to each cluster: [249 110 251]
n_init: 2 , Number of elements assigned to each cluster: [249 110 251]
n_init: 3 , Number of elements assigned to each cluster: [249 110 251]
n_init: 4 , Number of elements assigned to each cluster: [249 110 251]
n_init: 5 , Number of elements assigned to each cluster: [249 110 251]
n_init: 6 , Number of elements assigned to each cluster: [500 100  10]
n_init: 7 , Number of elements assigned to each cluster: [500 100  10]
n_init: 8 , Number of elements assigned to each cluster: [500 100  10]
n_init: 9 , Number of elements assigned to each cluster: [500 100  10]
n_init: 10 , Number of elements assigned to each cluster: [500 100  10]
True number of elements assigned to each cluster : [500 100  10]



* If we ask for `k` clusters the Kmeans will put the data into `k` clusters 
* **CAUTION** : `Non-optimal number of clusters`:<br>
In a real world setting there is no uniquely defined true number of clusters. An appropriate number of clusters has to be decided from data-based criteria and knowledge of the intended goal.

* `Anisotropically distributed blobs`: k-means consists of minimizing sample’s euclidean distances to the centroid of the cluster they are assigned to. As a consequence, k-means is more appropriate for clusters that are isotropic and normally distributed (i.e. spherical gaussians).

* `Unequal variance`: k-means is equivalent to taking the maximum likelihood estimator for a “mixture” of k gaussian distributions with the **same variances but with possibly different means**.

* `Unevenly sized blobs`: there is no theoretical result about k-means that states that it requires similar cluster sizes to perform well, yet **minimizing euclidean distances does mean that the more sparse and high-dimensional the problem is, the higher is the need to run the algorithm with different centroid seeds to ensure a global minimal inertia**.

#### In high-dimensional spaces, Euclidean distances tend to become inflated . Running a dimensionality reduction algorithm prior to k-means clustering can alleviate this problem and speed up the computations

#### In the case where clusters are known to be isotropic, have similar variance and are not too sparse, the k-means algorithm is quite effective and is one of the fastest clustering algorithms available. 

#### This advantage is lost if one has to restart it several times to avoid convergence to a local minimum.